In [ ]:
import numpy as np
import seaborn
import matplotlib.pyplot as plt
import pandas as pd

inds_to_data = pd.read_csv('edges.csv', header=None, names=['move', 'player', 'start', 'end', 'selected'])
players, player_indices = np.unique(inds_to_data['player'].to_numpy(), return_inverse=True)
inds_to_data['player'] = player_indices
moves = inds_to_data['move'].to_numpy()
starts = inds_to_data['start'].to_numpy()
ends = inds_to_data['end'].to_numpy()
ndots = len(np.unique(ends))

X = np.zeros((len(np.unique(moves)), len(np.unique(players)), ndots, ndots), dtype=bool)

X[moves-1, player_indices, starts-1, ends-1] = inds_to_data['selected'].to_numpy()

np.int64(40)